# THE COMMUNITY EARTH SYSTEM MODEL (CESM) LARGE ENSEMBLE PROJECT

Paper: https://journals.ametsoc.org/doi/pdf/10.1175/BAMS-D-13-00255.1

Authors: Kay et al. 2015

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#THE-COMMUNITY-EARTH-SYSTEM-MODEL-(CESM)-LARGE-ENSEMBLE-PROJECT" data-toc-modified-id="THE-COMMUNITY-EARTH-SYSTEM-MODEL-(CESM)-LARGE-ENSEMBLE-PROJECT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>THE COMMUNITY EARTH SYSTEM MODEL (CESM) LARGE ENSEMBLE PROJECT</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Create-Dask-Cluster" data-toc-modified-id="Create-Dask-Cluster-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Create Dask Cluster</a></span></li><li><span><a href="#Load-data-into-xarray-from-an-intake-esm-catalog" data-toc-modified-id="Load-data-into-xarray-from-an-intake-esm-catalog-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Load data into xarray from an intake-esm catalog</a></span></li><li><span><a href="#Get-Observation-data-(HadCRUT4;-Morice-et-al.-2012)" data-toc-modified-id="Get-Observation-data-(HadCRUT4;-Morice-et-al.-2012)-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Get Observation data (HadCRUT4; Morice et al. 2012)</a></span></li><li><span><a href="#Compute-areacella" data-toc-modified-id="Compute-areacella-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Compute areacella</a></span></li><li><span><a href="#Compute-weighted-spatial-means-for-model-data" data-toc-modified-id="Compute-weighted-spatial-means-for-model-data-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Compute weighted spatial means for model data</a></span></li><li><span><a href="#Compute-weighted-temporal-mean-for-obs-data" data-toc-modified-id="Compute-weighted-temporal-mean-for-obs-data-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Compute weighted temporal mean for obs data</a></span></li><li><span><a href="#Confirm-that-after-using-area-weighted-average,-max-temp-increase-is-5k" data-toc-modified-id="Confirm-that-after-using-area-weighted-average,-max-temp-increase-is-5k-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Confirm that after using area weighted average, max temp increase is 5k</a></span></li><li><span><a href="#Figure-2:-Global-surface-temperature-anomaly-(1961-90-base-period)-for-individual-ensemble-members,-and-observations" data-toc-modified-id="Figure-2:-Global-surface-temperature-anomaly-(1961-90-base-period)-for-individual-ensemble-members,-and-observations-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Figure 2: Global surface temperature anomaly (1961-90 base period) for individual ensemble members, and observations</a></span></li><li><span><a href="#Compute-Linear-Trend-for-boreal-winter-seasons-(DJF)" data-toc-modified-id="Compute-Linear-Trend-for-boreal-winter-seasons-(DJF)-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Compute Linear Trend for boreal winter seasons (DJF)</a></span></li><li><span><a href="#Figure-4:-Global-maps-of-historical-(1979---2012)-boreal-winter-(DJF)-surface-air-trends" data-toc-modified-id="Figure-4:-Global-maps-of-historical-(1979---2012)-boreal-winter-(DJF)-surface-air-trends-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Figure 4: Global maps of historical (1979 - 2012) boreal winter (DJF) surface air trends</a></span></li><li><span><a href="#Going-Further" data-toc-modified-id="Going-Further-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Going Further</a></span></li></ul></li></ul></div>

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import intake
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
import logging
logger = logging.getLogger("distributed.utils_perf")
logger.setLevel(logging.ERROR)

In [ ]:
import dask
dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})

## Learning Objectives

## Create Dask Cluster

In [ ]:
cluster = NCARCluster(memory="40GB")
cluster.adapt(minimum=1, maximum=200, wait_count=60)
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
col = intake.open_esm_metadatastore(collection_name='CESM1-LE')

In [ ]:
col

In [ ]:
col.df.head()

## Load data into xarray from an intake-esm catalog

In [ ]:
_, ds_20C = col.search(component='atm', stream='cam.h1', variable='TREFHT', experiment='20C').to_xarray(chunks={'time': 712}, override_coords=True).popitem()
_, ds_rcp = col.search(component='atm', stream='cam.h1', variable='TREFHT', experiment='RCP85').to_xarray(chunks={'time': 712}, override_coords=True).popitem()
t_20c = ds_20C['TREFHT']
t_rcp = ds_rcp['TREFHT']

In [ ]:
t_ref = t_20c.sel(time=slice('1961', '1990'))
t_ref

## Get Observation data (HadCRUT4; Morice et al. 2012)

In [ ]:
ds = xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/cru/hadcrut4/air.mon.anom.median.nc').load()
obs = ds['air']
obs

## Compute areacella


Area element: $\delta A = R^2 \delta \phi \delta \lambda \cos(\phi)$
    
where ϕ is latitude, δϕ is the spacing of the points in latitude, δλ is the spacing of the points in longitude, and R is Earth's radius. (In this formula, ϕ and λ are measured in radians.)

In [ ]:
R = 6.371e6
total_area = (4 * np.pi * R**2)
dϕ = np.radians((t_ref['lat'][1]-t_ref['lat'][0]).values)
dλ = np.radians((t_ref['lon'][1]-t_ref['lon'][0]).values)
dA = R**2 * np.abs(dϕ * dλ) * np.cos(np.radians(t_ref.lat))
areacella = dA * (0 * t_ref.isel(member_id=0, time=0) + 1)
areacella.plot()

In [ ]:
areacella.sum().values / total_area

## Compute weighted spatial means for model data

In [ ]:
t_ref_ts = ((t_ref.resample(time='AS').mean('time') * areacella)\
             .sum(dim=('lat', 'lon')) / total_area)\
             .mean(dim=('time', 'member_id'))
t_20c_ts = ((t_20c.resample(time='AS').mean('time') * areacella)\
            .sum(dim=('lat', 'lon'))) / total_area
t_rcp_ts = ((t_rcp.resample(time='AS').mean('time') * areacella)\
            .sum(dim=('lat', 'lon'))) / total_area

In [ ]:
t_ref_mean = t_ref_ts.load()
t_ref_mean

In [ ]:
t_20c_ts_df = t_20c_ts.to_series().unstack().T
t_20c_ts_df.head()

In [ ]:
t_rcp_ts_df = t_rcp_ts.to_series().unstack().T
t_rcp_ts_df.head()

## Compute weighted temporal mean for obs data

In [ ]:
ds.time_bnds

In [ ]:
time_bound_diff = ds.time_bnds.diff(dim='nbnds')[:, 0]
time_bound_diff

In [ ]:
wgts = time_bound_diff.groupby('time.year') / time_bound_diff.groupby('time.year')\
                      .sum(xr.ALL_DIMS)
wgts

In [ ]:
np.testing.assert_allclose(wgts.groupby('time.year').sum(xr.ALL_DIMS), 1.0)

In [ ]:
cond = obs.isnull()
ones = xr.where(cond, 0.0, 1.0)
obs_sum = (obs * wgts).resample(time='AS').sum(dim='time')
ones_out = (ones * wgts).resample(time='AS').sum(dim='time')
obs_s = (obs_sum / ones_out).mean(('lat', 'lon')).to_series()
obs_s.head()

In [ ]:
all_ts_anom = pd.concat([t_20c_ts_df, t_rcp_ts_df]) - t_ref_mean.data
years = [val.year for val in all_ts_anom.index]

## Confirm that after using area weighted average, max temp increase is 5k

In [ ]:
np.testing.assert_allclose(all_ts_anom.values.max(), 5.0, rtol=0.02)

## Figure 2: Global surface temperature anomaly (1961-90 base period) for individual ensemble members, and observations

![kay-et-al-2015-figure-2](../../../assets/kay-et-al-2015-figure-2.png)

In [ ]:
ax = plt.axes()

ax.tick_params(right=True, top=True, direction='out', length=6, width=2, grid_alpha=0.5)
ax.plot(years, all_ts_anom, color='grey')
ax.plot(years, all_ts_anom[1], color='black')
ax.plot(obs_s.index.year.tolist(), obs_s, color='red')

ax.text(0.3, 0.4, 'observations',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color='red', fontsize=10)
ax.text(0.3, 0.3, 'members 1-40',
        verticalalignment='bottom', horizontalalignment='left',
        transform=ax.transAxes,
        color='grey', fontsize=10)

ax.set_xticks([1850, 1920, 1950, 2000, 2050, 2100])
plt.ylim(-1, 5)
plt.xlim(1850, 2100)
plt.ylabel('Global Surface\nTemperature Anomaly (K)')
plt.show()

## Compute Linear Trend for boreal winter seasons (DJF)

In [ ]:
def linear_trend(da, dim='time'):
    da_chunk = da.chunk({dim: -1})
    trend = xr.apply_ufunc(calc_slope, da_chunk,
                           vectorize=True,
                           input_core_dims=[[dim]],
                           output_core_dims=[[]],
                           output_dtypes=[np.float],
                           dask='parallelized')
    return trend
    

def calc_slope(y):
    """ufunc to be used by linear_trend"""
    x = np.arange(len(y))
    return np.polyfit(x, y, 1)[0]

In [ ]:
# TODO - this should probably include only full seasons from 1979 and 2012
t = xr.concat([t_20c, t_rcp], dim='time')
seasons = t.sel(time=slice('1979', '2012')).resample(time='QS-DEC').mean('time').load()

In [ ]:
winter_seasons = seasons.sel(time=seasons.time.where(seasons.time.dt.month == 12, drop=True))
winter_trends = linear_trend(winter_seasons.chunk({'lat': 20, 'lon': 20, 'time': -1}))\
                .load() * len(winter_seasons.time)

In [ ]:
# TODO: this should be 34 I think, its not!
# assert len(winter_seasons.time) == 34  

In [ ]:
len(winter_seasons.time)

## Figure 4: Global maps of historical (1979 - 2012) boreal winter (DJF) surface air trends

![kay-et-al-2015-figure-4](../../../assets/kay-et-al-2015-figure-4.png)

In [ ]:
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(dpi=300)
levels = [-7, -6, -5, -4, -3, -2, -1, -0.5, 0, 0.5, 1, 2, 3, 4, 5, 6, 7]
fg = winter_trends.plot(col='member_id', col_wrap=4, transform=ccrs.PlateCarree(),
            subplot_kws={'projection': ccrs.Robinson(central_longitude=180)},
            add_colorbar=False, levels=levels, cmap='RdYlBu_r', extend='neither')

for ax in fg.axes.flat:
    ax.coastlines(color='grey')
    
# TODO: move the subplot title to lower left corners
# TODO: Add obs panel and ensemble mean at the end
    
fg.add_colorbar(orientation='horizontal')
fg.cbar.set_label('1979-2012 DJF surface air temperature trends (K/34 years)')
fg.cbar.set_ticks(levels)
fg.cbar.set_ticklabels(levels)
plt.show()

## Going Further